# Assessment 2: Path Following and Collision Avoiding
### Intro
This notebook is to be the record of completion for Assessment 2: Programming JetBots.
### Scenario
Building on the previous pest identification project, program a bot that can both follow a set course and respond when specific pests are located.

In [ ]:
# Pre-setup
# %pip install -Uqq ipywidgets
# %pip install -Uqq torch
# %pip install -Uqq cv2
# %pip install -Uqq jetbot
# %pip install -Uqq numpy

In [ ]:
# Setup and imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg, Robot
import math
import time

### **Road Following**   
- Upload the "*best_steering_model_xy_trt.pth*" model file obtained from the "live_demo_build_trt.ipynb" into this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy_trt.pth`` in this notebook's directory.    

### **Collision Avoidance**             
- Upload the "*best_model_trt.pth*" model file obtained from the "live_demo_resnet18_build_trt.ipnb" into this notebooks's directory. Once that's finished there should be a file named ``best_model_trt.pth`` in this notebook's directory.   
> **Note**: Collision Avoidance ``blocked`` class should include images of cars or any obstacle which you would expect to see on the road (It's good to start with one object first) meanwhile the ``free`` class should include background images of the road following track. 

Load the TRT optimized models by executing the cell below

In [ ]:


model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy.pth')) # well trained road following model

model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
model_collision.load_state_dict(torch.load('best_model.pth')) # well trained collision avoidance model

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model_collision = model_collision.to(device)

### Creating the Pre-Processing Function

We have now loaded our models, but there's a slight issue. The format that we trained our models on doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:


mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def preprocess_col(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [ ]:
camera = Camera()

In [ ]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
robot = Robot()

Now, we will define some sliders to control the JetBot
> **Note**: We have initialized the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control slider: To start your JetBot increase ``speed_control_slider`` 
2. Steering Gain slider: If you see your JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias slider: If you see your JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

4. Blocked slider: Display the probability in which there is an obstacle in the front of the Jetbot using the collision avoidance model
5. Time for stop slider: To manually set the time for which the jetbot should remain stopped after an object has been removed
6. Blocked threshold slider: To manually set the blocked threshold to stop the Jetbot after an object has been detected 

In [ ]:
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

display(image_widget)

display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network models for Road following and Collision Avoidance
3. Check an if statements which would allow the Jetbot to perform road following and stop whenever an obstacle has been detected 
4. Compute the approximate steering value
5. Control the motors using proportional / derivative control (PD)

In [ ]:
angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = speed_control_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
    image_preproc2 = preprocess_col(change['new']).to(device)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(model_collision(image_preproc2), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            xy = model(image_preproc2).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
            
    
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steer_val = pid + steer_bias 
    angle_last = angle
    robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0) 

execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower and collision avoider should work, but the neural network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow the track and avoid collisions effectively.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### Training the Jetbot:

You are required to use the NVIDIA Jetbot to train it to follow a predefined path within a specified environment, in this case the classroom following the pink tape.
You must document the entire training process, including code, model selection, and training parameters, in a Jupyter notebook.
The Jupyter notebook should include comments explaining the rationale behind their choices and any issues they encountered.
### Inference for Pest Detection:

Using the model you have created in Assessment 1 (or a similar one), you must implement inference to detect pests.
You need to demonstrate how the Jetbot can identify and respond to pests in real-time during its path following.
The Jupyter notebook should contain code for inference and comments explaining the model's performance.
### Critical Evaluation:

You must critically evaluate the accuracy and effectiveness of your trained model for pest detection during infernece.
You must offer insightful comments on areas where improvements can be made, such as model accuracy, inference speed, and generalizability.
The Jupyter notebook should include a section for this evaluation.
### Jupyter Notebook:

The Jupyter notebook should be well-organized and include clear explanations and comments where required to demonstrate the student's knowledge and thought process.
### Video Demonstration:

You must provide a short (less than 1-minute) video demonstrating the Jetbot in action (using Xbox game bar recording or simlilar). The video should showcase the path following and pest detection aspects.